In [ ]:
import os
import obspy
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense
from pathlib import Path
%matplotlib inline
%run helperfunctions.ipynb
os.getcwd()

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
os.chdir('lse_challenge/data')

In [ ]:
# Signal List
os.chdir('signal')
signals = []
signal_files = os.listdir()
signal_files = random.sample(signal_files, 400) #Need to expand for actual dataset
for sigfile in signal_files: 
    tmp_trace = obspy.read(sigfile)
    tmp_trace = tmp_trace.normalize()
    cutlength = np.random.randint(16, 150)
    trace_mat = [tmp_trace[0].data[cutlength:1000+cutlength], tmp_trace[1].data[cutlength:1000+cutlength], tmp_trace[2].data[cutlength:1000+cutlength]]
    #mag = np.sqrt(trace_mat[0]**2 + trace_mat[1]**2 + trace_mat[2]**2)
    #signals.append([trace_mat[0], trace_mat[1], trace_mat[2], mag, cutlength, 1])
    signals.append([trace_mat[0], cutlength, 1])
    signals.append([trace_mat[1], cutlength, 1])
    signals.append([trace_mat[2], cutlength, 1])
    
    # Negative Transformations
    neg_mat = rot_xyz(trace_mat, 0, -np.pi, 0)
    #signals.append([neg_mat[0], neg_mat[1], neg_mat[2], mag, cutlength, 1])
    signals.append([neg_mat[0], cutlength, 1])
    signals.append([neg_mat[1], cutlength, 1])
    signals.append([neg_mat[2], cutlength, 1])
    
    # X, Y, and Z Rotations by Theta
    theta = np.random.uniform()*2*np.pi
    x_mat = rot_x(trace_mat, theta)
    #signals.append([x_mat[0], x_mat[1], x_mat[2], mag, cutlength, 1])
    signals.append([x_mat[0], cutlength, 1])
    signals.append([x_mat[1], cutlength, 1])
    signals.append([x_mat[2], cutlength, 1])
    y_mat = rot_y(trace_mat, theta)
    #signals.append([y_mat[0], y_mat[1], y_mat[2], mag, cutlength, 1])
    signals.append([y_mat[0], cutlength, 1])
    signals.append([y_mat[1], cutlength, 1])
    signals.append([y_mat[2], cutlength, 1])
    z_mat = rot_z(trace_mat, theta)
    #signals.append([z_mat[0], z_mat[1], z_mat[2], mag, cutlength, 1])
    signals.append([z_mat[0], cutlength, 1])
    signals.append([z_mat[1], cutlength, 1])
    signals.append([z_mat[2], cutlength, 1])    
    
    # General Rotation by alpha, beta, gamma
    alpha = np.random.uniform()*2*np.pi
    beta = np.random.uniform()*2*np.pi
    gamma = np.random.uniform()*2*np.pi
    rot_mat = rot_xyz(trace_mat, alpha, beta, gamma)
    #signals.append([rot_mat[0], rot_mat[1], rot_mat[2], mag, cutlength, 1])
    signals.append([rot_mat[0], cutlength, 1])
    signals.append([rot_mat[1], cutlength, 1])
    signals.append([rot_mat[2], cutlength, 1])
    

# Noise List
os.chdir('../noise')
noise_files = os.listdir()
noise_files = random.sample(noise_files, 1200) #Need to expand for actual dataset
for noisefile in noise_files: 
    tmp_trace = obspy.read(noisefile)
    tmp_trace = tmp_trace.normalize()
    cutlength = np.random.randint(16, 150)
    trace_mat = [tmp_trace[0].data[cutlength:1000+cutlength], tmp_trace[1].data[cutlength:1000+cutlength], tmp_trace[2].data[cutlength:1000+cutlength]]
    #mag = np.sqrt(trace_mat[0]**2 + trace_mat[1]**2 + trace_mat[2]**2)
    #signals.append([trace_mat[0], trace_mat[1], trace_mat[2], mag, cutlength, 0])
    signals.append([trace_mat[0], cutlength, 0])
    signals.append([trace_mat[1], cutlength, 0])
    signals.append([trace_mat[2], cutlength, 0])
    
    # Negative Transformations
    neg_mat = rot_xyz(trace_mat, 0, -np.pi, 0)
    #signals.append([neg_mat[0], neg_mat[1], neg_mat[2], mag, cutlength, 0])
    signals.append([neg_mat[0], cutlength, 0])
    signals.append([neg_mat[1], cutlength, 0])
    signals.append([neg_mat[2], cutlength, 0])
    
    # X, Y, and Z Rotations by Theta
    theta = np.random.uniform()*2*np.pi
    x_mat = rot_x(trace_mat, theta)
    #signals.append([x_mat[0], x_mat[1], x_mat[2], mag, cutlength, 0])
    signals.append([x_mat[0], cutlength, 0])
    signals.append([x_mat[1], cutlength, 0])
    signals.append([x_mat[2], cutlength, 0])
    y_mat = rot_y(trace_mat, theta)
    #signals.append([y_mat[0], y_mat[1], y_mat[2], mag, cutlength, 0])
    signals.append([y_mat[0], cutlength, 0])
    signals.append([y_mat[1], cutlength, 0])
    signals.append([y_mat[2], cutlength, 0])
    z_mat = rot_z(trace_mat, theta)
    #signals.append([z_mat[0], z_mat[1], z_mat[2], mag, cutlength, 0])
    signals.append([z_mat[0], cutlength, 0])
    signals.append([z_mat[1], cutlength, 0])
    signals.append([z_mat[2], cutlength, 0])
    
    # General Rotation by alpha, beta, gamma
    alpha = np.random.uniform()*2*np.pi
    beta = np.random.uniform()*2*np.pi
    gamma = np.random.uniform()*2*np.pi
    rot_mat = rot_xyz(trace_mat, alpha, beta, gamma)
    #signals.append([rot_mat[0], rot_mat[1], rot_mat[2], mag, cutlength, 0])
    signals.append([rot_mat[0], cutlength, 0])
    signals.append([rot_mat[1], cutlength, 0])
    signals.append([rot_mat[2], cutlength, 0])
    
os.chdir('../')

sig_df = pd.DataFrame(signals, columns = ["trace", "cutlength", "signal"])
sig_df['p_arrival'] = 1000 - sig_df['cutlength']

## Training Neural Network for dectecting whether or not the trace has a signal

In [ ]:
# For rolling averages
roll_short = 25
roll_long = 50

# For location segments
window_step = 30
window_size = 30

In [ ]:
# Features and Targets for Identifying the Entire Trace

features = []
targets = []

for index, slice_df in sig_df.iterrows():
    # x = slice_df["tr_x"]
    # y = slice_df["tr_y"]
    # z = slice_df["tr_z"]
    tr = slice_df["trace"]
    mag = abs(tr)
    # mag = slice_df["magnitude"]
    signal = slice_df["signal"]
    p_arrival = slice_df["p_arrival"]
    #d = {"x": x, "y":y, "z":z, "magnitude":mag}
    d = {"trace": tr, "magnitude":mag}
    
    
    temp_df = pd.DataFrame(data = d)
    temp_df["STA"] = temp_df["magnitude"].rolling(roll_short).mean()
    temp_df["LTA"] = temp_df["magnitude"].rolling(roll_long).mean()
    temp_df["RAV"] = temp_df["STA"]/temp_df["LTA"]
    temp_df["STV"] = temp_df["magnitude"].rolling(roll_short).var()
    temp_df["LTV"] = temp_df["magnitude"].rolling(roll_long).var()
    
    temp_df.dropna(inplace = True)
    
    features.append(temp_df.values)
    targets.append(signal)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size = 0.3)

In [ ]:
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)
feature_shape = X_train[0].shape
feature_shape

In [ ]:
signalmodel = Sequential()
signalmodel.add(LSTM(32, input_shape = feature_shape))
signalmodel.add(Dense(1, activation = 'sigmoid'))
signalmodel.summary()

In [ ]:
signalmodel.compile(loss = 'binary_crossentropy', optimizer = "adam", metrics = ['accuracy'])
signalmodel.fit(X_train, y_train, epochs = 40)

In [ ]:
model_loss, model_accuracy = signalmodel.evaluate(X_test, y_test, verbose = 0)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
predictions_test = signalmodel.predict(X_test)
class_test = predictions_test > 0.50
f1_score_test = sklearn.metrics.f1_score(y_test, class_test)
print(f"F1 Score: {f1_score_test}")

In [ ]:
predictions_train = signalmodel.predict(X_train)
class_train = predictions_train > 0.50
f1_score_train = sklearn.metrics.f1_score(y_train, class_train)
print(f"F1 Score: {f1_score_train}")

In [ ]:
tn, fp, fn, tp = sklearn.metrics.confusion_matrix(y_test, class_test).ravel()
(tn, fp, fn, tp)

In [ ]:
# Exporting Model
os.getcwd()

In [ ]:
# Only run if we need to get to root directory
os.chdir('../../')

In [ ]:
# Exporting signal model
signalmodel.save('signalmodel')

## Training Neural Network for dectecting where in the trace the p-wave starts

In [ ]:
# Features and Targets for Identifying Windows of Trace
features = []
targets = []

for index, slice_df in sig_df.iterrows():
    
    signal = slice_df["signal"]
    
    if signal == 1:
        # x = slice_df["tr_x"]
        # y = slice_df["tr_y"]
        # z = slice_df["tr_z"]
        tr = slice_df["trace"]
        mag = abs(tr)
        # mag = slice_df["magnitude"]

        p_arrival = slice_df["p_arrival"]
        d = {"trace":tr, "magnitude":mag}

        temp_df = pd.DataFrame(data = d)
        temp_df["STA"] = temp_df["magnitude"].rolling(roll_short).mean()
        temp_df["LTA"] = temp_df["magnitude"].rolling(roll_long).mean()
        temp_df["RAV"] = temp_df["STA"]/temp_df["LTA"]
        temp_df["STV"] = temp_df["magnitude"].rolling(roll_short).var()
        temp_df["LTV"] = temp_df["magnitude"].rolling(roll_long).var()

        temp_df.dropna(inplace = True)

        start_ind = 0
        end_ind = start_ind + window_size



        while end_ind < (1000 - roll_long):
            # xwindow = temp_df["x"].iloc[start_ind:end_ind]
            # ywindow = temp_df["y"].iloc[start_ind:end_ind]
            # zwindow = temp_df["z"].iloc[start_ind:end_ind]
            trwindow = temp_df["trace"].iloc[start_ind:end_ind]
            magwindow = temp_df["magnitude"].iloc[start_ind:end_ind]
            # stawindow = temp_df["STA"].iloc[start_ind:end_ind]
            # ltawindow = temp_df["LTA"].iloc[start_ind:end_ind]
            ravwindow = temp_df["RAV"].iloc[start_ind:end_ind]
            stvwindow = temp_df["STV"].iloc[start_ind:end_ind]
            ltvwindow = temp_df["LTV"].iloc[start_ind:end_ind]

            window_data = {"trace": trwindow, "magnitude": magwindow,
                        "RAV": ravwindow, "STV": stvwindow, "LTV": ltvwindow}
            window_df = pd.DataFrame(data = window_data)

            features.append(window_df.values)

            if (p_arrival >= start_ind) and (p_arrival <= end_ind):
                targets.append(1)
            else:
                targets.append(0)

            start_ind += window_step
            end_ind = start_ind + window_size

In [ ]:
# For Windows of Trace
X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size = 0.3)

In [ ]:
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)
feature_shape = X_train[0].shape
feature_shape

In [ ]:
locmodel = Sequential()
locmodel.add(LSTM(32, input_shape = feature_shape))
locmodel.add(Dense(1, activation = 'sigmoid'))
locmodel.summary()

In [ ]:
locmodel.compile(loss = 'binary_crossentropy', optimizer = "adam", metrics = ['accuracy'])
locmodel.fit(X_train, y_train, epochs = 20)

In [ ]:
model_loss, model_accuracy = locmodel.evaluate(X_test, y_test, verbose = 0)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
predictions_test = locmodel.predict(X_test)
class_test = predictions_test > 0.25
f1_score_test = sklearn.metrics.f1_score(y_test, class_test)
print(f"F1 Score: {f1_score_test}")

In [ ]:
predictions_train = locmodel.predict(X_train)
class_train = predictions_train > 0.25
f1_score_train = sklearn.metrics.f1_score(y_train, class_train)
print(f"F1 Score: {f1_score_train}")

In [ ]:
tn, fp, fn, tp = sklearn.metrics.confusion_matrix(y_test, class_test).ravel()
(tn, fp, fn, tp)

In [ ]:
# Exporting location model
locmodel.save('locmodel')

## Spot checking results

In [ ]:
loaded_signalmodel = load_model('signalmodel')

In [ ]:
loaded_locmodel = load_model('locmodel')

In [ ]:
slice_df = sig_df.iloc[29]

In [ ]:
sigfeatures = []
sigtargets = []
locfeatures = []
loctargets = []

# x = slice_df["tr_x"]
# y = slice_df["tr_y"]
# z = slice_df["tr_z"]
tr = slice_df["trace"]
mag = abs(tr)
# mag = slice_df["magnitude"]
signal = slice_df["signal"]
p_arrival = slice_df["p_arrival"]
#d = {"x": x, "y":y, "z":z, "magnitude":mag}
d = {"trace": tr, "magnitude":mag}
temp_df = pd.DataFrame(data = d)

In [ ]:
temp_df["STA"] = temp_df["magnitude"].rolling(roll_short).mean()
temp_df["LTA"] = temp_df["magnitude"].rolling(roll_long).mean()
temp_df["RAV"] = temp_df["STA"]/temp_df["LTA"]
temp_df["STV"] = temp_df["magnitude"].rolling(roll_short).var()
temp_df["LTV"] = temp_df["magnitude"].rolling(roll_long).var()
    
temp_df.dropna(inplace = True)
sigfeatures.append(temp_df.values)
sigtargets.append(signal)

In [ ]:
loaded_signalmodel.predict(np.array(sigfeatures))

In [ ]:
sigtargets

In [ ]:
start_ind = 0
end_ind = start_ind + window_size
    
while end_ind < (1000 - roll_long):
    # xwindow = temp_df["x"].iloc[start_ind:end_ind]
    # ywindow = temp_df["y"].iloc[start_ind:end_ind]
    # zwindow = temp_df["z"].iloc[start_ind:end_ind]
    magwindow = temp_df["magnitude"].iloc[start_ind:end_ind]
    trwindow = temp_df["trace"].iloc[start_ind:end_ind]
    ravwindow = temp_df["RAV"].iloc[start_ind:end_ind]
    stvwindow = temp_df["STV"].iloc[start_ind:end_ind]
    ltvwindow = temp_df["LTV"].iloc[start_ind:end_ind]

    window_data = {"trace":trwindow, "magnitude": magwindow,
                    "RAV": ravwindow, "STV": stvwindow, "LTV": ltvwindow}
        
    window_df = pd.DataFrame(data = window_data)

    locfeatures.append(window_df.values)

    if (p_arrival >= start_ind) and (p_arrival <= end_ind):
        loctargets.append(1)
    else:
        loctargets.append(0)

    start_ind += window_step
    end_ind = start_ind + window_size

In [ ]:
loaded_locmodel.predict(np.array(locfeatures))

In [ ]:
loctargets

In [ ]:
## Pseudo Coding Test Script
"""
1) Take in trace file
2) Normalize data
3) Calculate Fields for signalmodel
4) Predict whether or not the trace contains a signal

If YES:
5) Calculate Fields for locmodel
6) Predict where the pwave is
7) Take the max probability
8) Convert to a UTC timestamp
9) Return UTC timestamp

If NO:
5) Return None
"""